In [4]:
# FILEPATH: Jupyter Notebook Cell
!conda init --all
# !conda create --name iCLIP_hackathon python=3.10
!conda activate iCLIP_hackathon
# !conda deactivate


no change     /home2/2459810a/miniconda3/condabin/conda
no change     /home2/2459810a/miniconda3/bin/conda
no change     /home2/2459810a/miniconda3/bin/conda-env
no change     /home2/2459810a/miniconda3/bin/activate
no change     /home2/2459810a/miniconda3/bin/deactivate
no change     /home2/2459810a/miniconda3/etc/profile.d/conda.sh
no change     /home2/2459810a/miniconda3/etc/fish/conf.d/conda.fish
no change     /home2/2459810a/miniconda3/shell/condabin/Conda.psm1
no change     /home2/2459810a/miniconda3/shell/condabin/conda-hook.ps1
no change     /home2/2459810a/miniconda3/lib/python3.10/site-packages/xontrib/conda.xsh
no change     /home2/2459810a/miniconda3/etc/profile.d/conda.csh
no change     /home2/2459810a/.bashrc
no change     /home2/2459810a/.zshrc
no change     /home2/2459810a/.config/fish/config.fish
no change     /home2/2459810a/.xonshrc
no change     /home2/2459810a/.tcshrc
No action taken.

CommandNotFoundError: Your shell has not been properly configured to use 'conda 

Various genome-related files are required during the iCLIP analysis pipeline.
The steps to generate these should be the same across all datasets analysed, if the virus is the same, so the genome directory can be copied into the new analysis folder without needing to rerun this.

Sources of data used are:

Human annotation: http://ftp.ensembl.org/pub/release-106/gtf/homo_sapiens/
Homo_sapiens.GRCh38.106.gtf.gz

Human fasta: http://ftp.ensembl.org/pub/release-106/fasta/homo_sapiens/dna/ Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz

SINV GTF:


In [1]:
# Merge human and SINV fasta files. Viral genome will represent an additional 'chromosome' in the genome. The fasta header used here should be the chromosome name (seqname) used in the annotation.

!cd /home2/2459810a/iCLIP_analysis/PURAB_hackathon_test/GENOMEDIR
!cat Homo_sapiens.GRCh38.dna.primary_assembly.fa SINV_fasta.fa | \
!gzip > HS.GRCh38.SINV.fa.gz


cat: Homo_sapiens.GRCh38.dna.primary_assembly.fa: No such file or directory
cat: SINV_fasta.fa: No such file or directory
/bin/bash: !gzip: command not found


In [5]:
!cd /home2/2459810a/PURAB_hackathon_test/
!mkdir -p Data/Merged

!cat \
/home5/castello_group/iCLIP_analysis/PURAB_analysis/Data/iClip2_Mock_200221_S0_L001_R1_001.fastq.gz \
/home5/castello_group/iCLIP_analysis/PURAB_analysis/Data/iClip2_Mock_200221_S0_L002_R1_001.fastq.gz \
/home5/castello_group/iCLIP_analysis/PURAB_analysis/Data/iClip2_Mock_200221_S0_L003_R1_001.fastq.gz \
/home5/castello_group/iCLIP_analysis/PURAB_analysis/Data/iClip2_Mock_200221_S0_L004_R1_001.fastq.gz >\
Data/Merged/PURAB_iCLIP_RAW.fastq.gz

In [ ]:
## Remove PhiX Reads

!/software/bbmap-v38.90/bbduk.sh \
in=Data/Merged/PURAB_iCLIP_RAW.fastq.gz \
out=Data/Merged/PURAB_iCLIP_rmPhiX.fastq.gz \
ref=/software/bbmap-v38.90/resources/phix174_ill.ref.fa.gz \
k=31 hdist=1 -Xmx16g threads=2 qin=33 \
stats=Data/Merged/PURAB_iCLIP_rmPhiX.stats.txt

In [ ]:
## QC Sequencing Runs

### Run FastQC

# This is for all of the reads combined 
# The parts to focus on are: 
# *per base sequence quality - this can sometimes drop towards the 3' end. In this case, 3' ends of sequences can be trimmed. 
# *per base sequence content - biases in positions 6 to 12 will be determined by the barcodes positions 1-5 and 13-16 will reflect UMI composition


!mkdir -p Data/Merged/FastQC

files=!ls Data/Merged/*_rmPhiX.fastq.gz

for file in files:
    print("Processing file", file)
    !fastqc $file -o Data/Merged/FastQC

In [22]:
# Conda installation requirements 
# conda install -c bioconda fastqc
!conda install -c bioconda multiqc

Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: unsuccessful attempt using repodata from current_repodata.json, retrying with next repodata source.

ResolvePackageNotFound: 
  - conda==23.3.1



In [25]:
### Produce MultiQC report (FastQC)



!mkdir -p Plots/QC

!multiqc -f \
Data/Merged/FastQC/*_fastqc.zip \
-o Plots/QC \
-n 01_Sequencing-runs_MultiQC-report



/bin/bash: multiqc: command not found


#### quality filter on barcode region??? why not have this???

## Demultiplex

Check the frequency of barcodes

This returns frequency of all barcodes in the dataset It is important to
check that all barcodes are the top hits and also that there aren't any
additional, over-represented barcodes In an uncontaminated library,
expected barcodes should outnumber other barcodes by at least 10-fold
The most frequently occurring unexpected barcodes should only differ
slightly from barcode seqs - these are probably a result of
amplification/ sequencing errors in the barcode region

In [27]:
!zcat Data/Merged/PURAB_iCLIP_rmPhiX.fastq.gz | awk -v umi1_len=5 -v exp_bc_len=6 '{if (FNR%4==2) print substr($1,(umi1_len+1),exp_bc_len)}' | sort | uniq -c | sort -k1,1rn>barcode_freqs


Barcode file should be a txt file with 2 columns: sample_name barcode

Sample names can be whatever you want but it will make downstream steps
simpler if it contains: - Gene name (GFP/gene of interest) - Treatment
condition (mock/SINV) - Sample type (SMI/IP) - Replicate n

e.g. DDX1_SINV_IP_rep1

Barcodes should be in the format NNNNNCTGATCNNNN, with 5 Ns before the
barcode and 4 after. The barcode sequence should be the reverse
complement of the barcode!!! The Ns are because of the structure of
iCLIP barcodes, which contain UMI sequences that the demultiplexing tool
accounts for.

In [ ]:
!mkdir -p Trimming/Demultiplex_0MM

!java -jar /software/je_1.2/je_1.2_bundle.jar demultiplex \
F1=Data/Merged/PURAB_iCLIP_rmPhiX.fastq.gz \
BF=PURAB_barcodes_rc_umi.txt \
RCHAR=':' \
O=Trimming/Demultiplex \
UF1=Unassigned_PURAB_iCLIP_merged_jemultiplexer.fastq \
M=PURAB_iCLIP_merged_jemultiplexer_out_stats.txt \
FASTQ_FILE_EXTENSION=fastq \
MM=0

In [ ]:
## QC Individual Libraries

### Run FastQC

!mkdir -p Trimming/Demultiplex/FastQC

for file in Trimming/Demultiplex/*.fastq.gz
do
echo "Processing file $file"
fastqc $file -o Trimming/Demultiplex/FastQC
echo ""
done 

# ADAPTER TRIMMING

## Trim Adapters With Cutadapt

Remove the adapter sequence on all of the reads and remove any sequences that are too short.

NOTE: Barcode unassigned files are skipped

In [ ]:
mkdir -p Trimming/Adapter

for file in Trimming/Demultiplex/*.fastq.gz
do
[[ $file == *Unassigned*.fastq.gz ]] && continue 
echo "" 
echo "Processing file $file" 
base=$(basename "$file") 
outfile="$(echo ${base} |sed -e 's/.fastq.gz/_trimmed.fastq.gz/')" 
outshort="$(echo ${base} |sed -e 's/.fastq.gz/_trimmed.fastq.tooshort.gz/')" 
outinfo="$(echo ${base} |sed -e 's/.fastq.gz/_trimmed.info/')" 
cutadapt $file \
-a AGATCGGAAGAGCGGTTCAG \
-j 4 -e 0.1 -O 1 --nextseq-trim 10 --minimum-length 15 \
-o Trimming/Adapter/$outfile \
--too-short-output Trimming/Adapter/$outshort > \
Trimming/Adapter/$outinfo 
echo "" 
done

In [ ]:
## Produce MultiQC report (Cutadapt)


mkdir -p Plots/QC 

multiqc -f \
Trimming/Adapter/*_trimmed.info \
-o Plots/QC \
-n 03_Cutadapt_MultiQC-report


## QC rRNA Contamination

### Map to human rDNA with bbduk

One thing that can reduce frequency of aligned reads is rRNA
contamination so it's good to check what proportion of reads map to
ribosomal RNA. Make sure that the rDNA fasta file is present in the
analysis directory. This fasta can be downloaded from:
<https://www.ncbi.nlm.nih.gov/nuccore/U13369.1?report=fasta>

rRNA reads can also be a reflection of true binding but they will be removed during alignment as they do not uniquely map. It is useful to have an idea of mapping, therefore, as this information will be lost otherwise.


In [29]:
mkdir -p Trimming/rRNA

for file in Trimming/Adapter/*.fastq.gz
do
echo ""
base=$(basename $file .fastq.gz)
echo "Processing $file"
/software/bbmap-v38.90/bbduk.sh \
-Xmx24G \
in=$file \
out=Trimming/rRNA/$(echo ${base}).minus.rRNA.fastq.gz \
ref=Hsap_rDNA_U13369.1.fa \
k=25 \
stats=Trimming/rRNA/$(echo ${base}).rRNA.stat.txt \
hdist=1
echo ""
done


SyntaxError: invalid syntax (1251040940.py, line 1)

## Run STAR

The original script had: \### Gunzip fastq.gz for NTFS system to prevent
FIFO file writing for file in Trimming/Adapter/\*\_trimmed.fastq.gz do
base=$(basename "$file" ) output="Trimming/Adapter/\$(echo \${base}
\|sed -e 's/.gz//')" gunzip -c -f \$file done

For me, this just unzipped all the files and printed them to the screen
as far as I could tell...

Instead, I unzipped the files. This seemed to work so not sure
what the above script was written to fix.

In [ ]:
for file in Trimming/Adapter/*_trimmed.fastq.gz 
do 
gunzip -f $file 
done

In [ ]:
# Creating STAR object

STAR --runThreadN 8 --runMode genomeGenerate \
--genomeFastaFiles HS.GRCh38.SINV.fa \
--sjdbGTFfile Homo_sapiens.GRCh38.106_SINV.gtf \
--genomeDir STAR \
--alignSJoverhangMin 8 --sjdbOverhang 100 \
--limitGenomeGenerateRAM 31000000000 --genomeSAindexNbases 10 --genomeSAsparseD 4

In [ ]:
### Align with STAR

```{bash}
mkdir -p Alignment 

for file in Trimming/Adapter/*_trimmed.fastq 
do 
echo "" 
echo "Processing file $file" 
base=$(basename "$file") 
outdir="$(echo ${base} |sed -e 's/_trimmed.fastq//')" 
mkdir Alignment/$outdir 
STAR --runMode alignReads --runThreadN 8 \
--outSAMtype BAM SortedByCoordinate \
--genomeDir GENOMEDIR/STAR \
--sjdbGTFfile GENOMEDIR/Homo_sapiens.GRCh38.106_HIV.modified.gtf \
--readFilesIn $file --outFileNamePrefix Alignment/$outdir/ \
--outReadsUnmapped Fastx \
--outFilterMismatchNmax 999 \
--outFilterMultimapNmax 1 \
--outFilterMismatchNoverLmax 0.04 \
--outSJfilterReads Unique \
--alignEndsType EndToEnd
echo "" 
done 

```

### Produce MultiQC report (STAR)

```{bash, engine.opts = '-l'}
mkdir -p Plots/QC 

multiqc -f \
Alignment/*/*Log.final.out \
-o Plots/QC \
-n 05_STAR_MultiQC-report

```

### Index bam files and produce idxstats- pre-dedup

```{bash, engine.opts = '-l'}
mkdir -p Alignment_idxstats

for dir in $(ls -d Alignment/*/) 
do
echo "" 
file=$dir/Aligned.sortedByCoord.out.bam 
base=$(basename "$dir") 
output="Alignment_idxstats/$(echo ${base}).idxstats.txt"  
echo "Processing file $file" 
samtools index $file 
samtools idxstats $file > $output 
done

```

### Produce MultiQC report (idxstat)

This is pre-deduplication

```{bash, engine.opts = '-l'}
mkdir -p Plots/QC

multiqc -f \
Alignment_idxstats/*idxstats.txt \
-o Plots/QC \
-n 06_Chromosome-mapping-pre-dedup_MultiQC-report

```

# COLLAPSE UMI

## Deduplicate using UMIs (je markdupes)

```{bash}
mkdir -p Dedup 

for dir in $(ls -d Alignment/*/) 
do
echo "" 
input=$dir/Aligned.sortedByCoord.out.bam 
base=$(basename "$dir") 
output="Dedup/$(echo ${base}).dedup.je.bam" 
outlog="Dedup/$(echo ${base}).dedup.je.log" 
metrics="Dedup/$(echo ${base}).dedup.je.metrics" 
java -jar /software/je_1.2/je_1.2_bundle.jar markdupes \
I=$input \
O=$output \
M=$metrics \
REMOVE_DUPLICATES=True \
MM=1 >\
$outlog
echo "" 
done

```

### Index bam files and produce idxstats - post-dedup

```{bash, engine.opts = '-l'}
mkdir -p Dedup_idxstats

for file in $(ls Dedup/*.dedup.je.bam) 
do
echo "" 
echo "Processing file $file" 
base=$(basename "$file") 
output="Dedup_idxstats/$(echo ${base} |sed -e 's/.dedup.je.bam/.idxstats.txt/')" 
samtools index "$file" 
samtools idxstats $file > $output
done

```

### Produce multiQC report (idxstat)

This is post-deduplication

```{bash, engine.opts = '-l'}
mkdir -p Plots/QC 

multiqc -f \
Dedup_idxstats/*idxstats.txt \
-o Plots/QC \
-n 07_Chromosome-mapping-post-dedup_MultiQC-report

```

# ASSESS READ RECOVERY

## Collect Read Count Data

### Raw read counts

the file is in the gz format so has to be unzipped before reading. zcat
unzips the file within the operation but not outside of it (the file
remains stored as .gz) wc -l gives the line count this is divided by 4
since there are 4 lines per entry \| bc is 'basic calculator' so to do
1+2 in linux, you do 1+2 \| bc

```{bash, engine.opts = '-l'}
outfile='Trimming/Demultiplex/Raw-read-counts.txt' 

for file in $(ls Trimming/Demultiplex/*.fastq.gz)
do
echo "File is $file"
base=$(basename "$file" .fastq.gz) 
count=$(echo $(zcat $file | wc -l) / 4 | bc) 
echo "Count is $count"
echo "$base $count">> $outfile
done

```

### Trimmed read counts

```{bash, engine.opts = '-l'}
outfile='Trimming/Adapter/Trimmed-read-counts.txt' 

for file in $(ls Trimming/Adapter/*.fastq)
do
echo "File is $file"
chmod 744 $file
base=$(basename "$file" _trimmed.fastq) 
count=$(echo $($file | wc -l) / 4 | bc) 
echo "Count is $count"
echo "$base $count">> $outfile
done

outfile='Trimming/Adapter/Trimmed-read-counts.txt' 

for file in $(ls Trimming/Adapter/*.fastq)
do
echo "File is $file"
base=$(basename "$file" _trimmed.fastq) 
count=$(echo $(expr $(cat $file | wc -l) / 4)) 
echo "Count is $count"
echo "$base $count">> $outfile
done

```

### Uniquely mapped read counts

```{bash, engine.opts = '-l'}
outfile='Alignment/Unique-map-read-counts.txt'

for dir in $(ls -d Alignment/*/) 
do
echo "" 
file=$dir/Aligned.sortedByCoord.out.bam 
base=$(basename "$dir") 
echo "File is $file" 
count=$(samtools view -c $file) 
echo "Count is $count"
echo "$base $count" >> $outfile
done

```

### Deduplicated read counts

```{bash, engine.opts = '-l'}
outfile='Dedup/Dedup-read-counts.txt'

for file in $(ls Dedup/*.dedup.je.bam) 
do
echo "" 
echo "File is $file" 
base=$(basename "$file" .dedup.je.bam) 
count=$(samtools view -c $file) 
echo "Count is $count"
echo "$base $count" >> $outfile
done

```

# CROSSLINKED SITES

These steps generate bigwig files, which are useful for determining peak shape and visualising the data. They don't, however, contribute to the core pipeline.

## Get Xlsite

### Collapse reads to the 5' position (I don't think this does this...)
The 'collapsed' read is just the bam converted to bed 
Shift just shifts everything back by one position 
  The genome size file, which contains info on the length of every chromosome 


```{bash, engine.opts = '-l'}

mkdir -p Xlsite/collapsed
mkdir -p Xlsite/shifted/bed

for file in Dedup/*.dedup.je.bam 
do
echo "Processing $file" 
base=$(basename $file) 
outbed="Xlsite/collapsed/$(echo ${base} |sed -e 's/.dedup.je.bam/.collapsed.bed/')" 
outbedshifted="Xlsite/shifted/bed/$(echo ${base} |sed -e 's/.dedup.je.bam/.shifted.bed/')" 
bedtools bamtobed -i $file > $outbed 
bedtools shift -m 1 -p -1 -i $outbed -g GENOMEDIR/sizes.genome > $outbedshifted 
echo ""
done

```

## Create Bedgraphs (RPM) For Browsing
i assume we do not require shifted for our HIV data (rozeena)

```{bash, engine.opts = "-l"}
mkdir -p Xlsite/shifted/bedgraph

for file in Xlsite/shifted/bed/*.shifted.bed 
do 
echo "Processing $file" 
base=$(basename $file .shifted.bed) 
outbgplus="Xlsite/shifted/bedgraph/$(echo ${base}).XL.RPM.+.bedgraph" 
outbgminus="Xlsite/shifted/bedgraph/$(echo ${base}).XL.RPM.-.bedgraph" 
TmpScale=$(bc <<< "scale=6;1000000/$(awk -v var="$base" '$1 == var { print $2}' Dedup/Dedup-read-counts.txt)") 
bedtools genomecov -bg -strand + -5 -i $file -g GENOMEDIR/sizes.genome -scale $TmpScale > $outbgplus 
bedtools genomecov -bg -strand - -5 -i $file -g GENOMEDIR/sizes.genome -scale $TmpScale > $outbgminus 
echo ""
done
```

## Convert Bedgraph To BigWig

### Sort bedgraphs

```{bash, engine.opts = "-l"}
mkdir -p Xlsite/shifted/bedgraph_sorted

LC_COLLATE=C

for file in Xlsite/shifted/bedgraph/*.bedgraph 
do 
echo "Processing $file" 
base=$(basename $file .bedgraph) 
output="Xlsite/shifted/bedgraph_sorted/$(echo ${base}).sorted.bedgraph" 
sort -k1,1 -k2,2n $file > $output
echo ""
done

```

### Convert to Bigwig

```{bash, engine.opts = "-l"}
mkdir -p Xlsite/shifted/bw

for file in Xlsite/shifted/bedgraph_sorted/*.bedgraph 
do 
echo "Processing $file" 
base=$(basename $file .sorted.bedgraph) 
output="Xlsite/shifted/bw/$(echo ${base}).bw" 
/software/kentUtils-v302.1.0/bin/linux.x86_64/bedGraphToBigWig –switches $file GENOMEDIR/sizes.genome $output
echo ""
done

```

\newpage

# HTSEQ-CLIP

## Extract Crosslinks

Crosslink sites are extracted one position upstream of the read end as this is, theoretically, where the RT will have stopped.

```{bash}

mkdir -p DEW-seq/Extract

for file in Dedup/*.dedup.je.bam
do
echo "File is $file"
base=$(basename "$file")
output="DEW-seq/Extract/$(echo ${base} |sed -e 's/.dedup.je.bam/.extract.bed/')"
/home4/2711498i/miniconda3/envs/htseq-clip/bin/htseq-clip extract \
-i $file \
-e 1 \
-s s \
-g -1 \
--ignore \
-o $output
echo ""
done

```

## Creating flattened genome annotation (trial by Rozeena)
## dont run this code snippet

```{bash}

gff_file="GENOMEDIR/Homo_sapiens.GRCh38.106_HIV.gtf"
echo "File is $gff_file"

output="GENOMEDIR/Homo_sapiens.GRCh38.106_HIV.gtf.bed"
/home4/2711498i/miniconda3/envs/htseq-clip/bin/htseq-clip annotation \
-g $gff_file \
-t gene_biotype \
--splitExons \
-o $output


```




## Count Against Sliding Windows

This requires the flattened genome annotation with the desired window size. If window size needs to be altered, this has to be done at the genome annotation level.

```{bash}
# 50/20
mkdir -p DEW-seq/Count_w50s20

for file in DEW-seq/Extract/*.bed
do
echo "File is $file"
base=$(basename "$file")
output="DEW-seq/Count_w50s20/$(echo ${base} |sed -e 's/.extract.bed/.count.bed/')"
/home4/2711498i/miniconda3/envs/htseq-clip/bin/htseq-clip count \
-i $file \
-a GENOMEDIR/sliding_window/Homo_sapiens.GRCh38.106_HIV.modified.flattened.w50s20.txt.gz \
-o $output
echo ""
done

```

## Create Matrix

Combine all htseq results into a single file

```{bash}
mkdir -p DEW-seq/Matrix_w50s20

/home4/2711498i/miniconda3/envs/htseq-clip/bin/htseq-clip createMatrix \
-i DEW-seq/Count_w50s20 \
-e .bed \
-o DEW-seq/Matrix_w50s20/Full_iCLIP.matrix.txt.gz

```

# DEW-SEQ

## R Environment

From this point onwards, the analysis is done using DEWseq in R.

```{r}
library(tidyverse)
library(data.table)
library(IHW)
library(DEWSeq)
library(ggrepel)
library(BiocParallel)
library(scales)

```

```{r}
# * * * * * Data

## Read data
countData <- fread("/home2/2459810a/PURAB_Analysis/DEW-seq/Matrix_w50s20/Full_iCLIP.matrix.txt.gz", sep = "\t")
annotationData <- fread("/home2/2459810a/PURAB_Analysis/GENOMEDIR/sliding_window/Homo_sapiens.GRCh38.106_HIV.modified.flattened.w50s20.maptoid.txt.gz", sep = "\t")


extract_regions <- function(resultWindows, padjThresh, log2FoldChangeThresh, trackName, fileName) {
+   resultWindows <- as.data.frame(resultWindows)
+   resultRegions <- extractRegions(windowRes  = resultWindows,
+                                   padjCol    = "p_adj_IHW",
+                                   padjThresh = padjThresh,
+                                   log2FoldChangeThresh = log2FoldChangeThresh)+ toBED(windowRes = resultWindows,
+         regionRes = resultRegions,
+         padjCol    = "p_adj_IHW",
+         padjThresh = padjThresh,
+         log2FoldChangeThresh = log2FoldChangeThresh,
+         trackName = trackName,
+         fileName  = fileName
+   )
+   return(resultRegions)
+ }
> 


# * * * * * PURA

##
PURA <- countData %>%
  dplyr::select(unique_id, contains("PURA"), -contains("GFP")) %>%
  dplyr::select(unique_id, contains("IP"), contains("SMI")) %>%
  column_to_rownames(var = "unique_id")

PURA_colData <- tibble(name = colnames(PURA)) %>%
  mutate(type = case_when(
    str_detect(name, "IP") ~ "IP",
    str_detect(name, "SMI") ~ "SMI"
  )) %>%
  mutate(type = fct_relevel(type, c("IP", "SMI"))) %>%
  column_to_rownames(var = "name")

##
PURA_ddw <- DESeqDataSetFromSlidingWindows(countData = PURA,
                                            colData = PURA_colData,
                                            annotObj = annotationData,
                                            tidy = FALSE,
                                            design = ~type,
                                            start0based = TRUE)

##
keep <- rowSums(counts(PURA_ddw)) >= 5
PURA_ddw <- PURA_ddw[keep,]
rm(list = "keep")

PURA_ddw <- estimateSizeFactors(PURA_ddw)
sizeFactors(PURA_ddw)

PURA_ddw_mRNAs <- PURA_ddw[ rowData(PURA_ddw)[,"gene_type"] == "protein_coding", ]
PURA_ddw_mRNAs <- estimateSizeFactors(PURA_ddw_mRNAs)
sizeFactors(PURA_ddw) <- sizeFactors(PURA_ddw_mRNAs)
sizeFactors(PURA_ddw)

PURA_ddw_tmp <- PURA_ddw
PURA_ddw_tmp <- estimateDispersions(PURA_ddw_tmp, fitType = "local", quiet = TRUE)
PURA_ddw_tmp <- nbinomWaldTest(PURA_ddw_tmp)
tmp_significant_windows <- 
  results(PURA_ddw_tmp,
          contrast = c("type", "IP", "SMI"),
          parallel = FALSE,
          BPPARAM = 4,
          tidy = TRUE,
          filterFun = ihw) %>% 
  dplyr::filter(padj < 0.05) %>% 
  .[["row"]]
rm("PURA_ddw_tmp")

PURA_ddw_mRNAs <- PURA_ddw_mRNAs[ !rownames(PURA_ddw_mRNAs) %in% tmp_significant_windows, ]
PURA_ddw_mRNAs <- estimateSizeFactors(PURA_ddw_mRNAs)
sizeFactors(PURA_ddw) <- sizeFactors(PURA_ddw_mRNAs)
sizeFactors(PURA_ddw)
rm( list = c("tmp_significant_windows", "PURA_ddw_mRNAs"))

## 
PURA_ddw <- estimateDispersions(PURA_ddw, fitType = "local", quiet = TRUE)
PURA_ddw <- nbinomWaldTest(PURA_ddw)

PURA_resultWindows <- resultsDEWSeq(PURA_ddw,
                                     contrast = c("type", "IP", "SMI"),
                                     parallel = FALSE,
                                     BPPARAM = 12,
                                     tidy = TRUE) %>% as_tibble

PURA_resultWindows[,"p_adj_IHW"] <- adj_pvalues(ihw(pSlidingWindows ~ baseMean, 
                                                     data = PURA_resultWindows,
                                                     alpha = 0.05,
                                                     nfolds = 10))

saveRDS(PURA_ddw, "./DEW-seq/Matrix_w50s20/PURA_ddw_ct5.RDS")
saveRDS(PURA_resultWindows, "./DEW-seq/Matrix_w50s20/PURA_resultWindows.RDS")

##
PURA_resultWindows %>%
  filter(p_adj_IHW < 0.05) %>%
  filter(log2FoldChange > 2) %>%
  filter(chromosome == "NC_045512.2") %>%
  arrange(desc(log2FoldChange)) %>% pull(gene_name) %>% unique() %>% head(20)

##
pdf("./Plots/DEW-seq_Dispersion_PURA_ct5.pdf", width = 7, height = 7)
plotDispEsts(PURA_ddw)
dev.off()

#extract Regions

extract_regions(resultWindows = PURA_resultWindows,
	padjThresh = 0.01,
	log2FoldChangeThresh = 2,
	trackName = "Binding Site HIV_PURA HIV 0.01",
	fileName = "./DEW-seq/Regions_w50s20/HIV_PURA_enrichedregions.log2FC_2.0_p0.01.bed") %>%
	saveRDS("./DEW-seq/Regions_w50s20/HIV_PURA_enrichedregions.log2FC_2.0_p0.01.RDS")


extract_regions(resultWindows = PURA_resultWindows,
	padjThresh = 0.05,
	log2FoldChangeThresh = 2,
	trackName = "Binding Site HIV_PURA HIV 0.05",
	fileName = "./DEW-seq/Regions_w50s20/HIV_PURA_enrichedregions.log2FC_2.0_p0.05.bed") %>%
	saveRDS("./DEW-seq/Regions_w50s20/HIV_PURA_enrichedregions.log2FC_2.0_p0.05.RDS")


# * * * * * PURB
##
PURB <- countData %>%
  dplyr::select(unique_id, contains("PURB"), -contains("GFP")) %>%
  dplyr::select(unique_id, contains("IP"), contains("SMI")) %>%
  column_to_rownames(var = "unique_id")

PURB_colData <- tibble(name = colnames(PURB)) %>%
  mutate(type = case_when(
    str_detect(name, "IP") ~ "IP",
    str_detect(name, "SMI") ~ "SMI"
  )) %>%
  mutate(type = fct_relevel(type, c("IP", "SMI"))) %>%
  column_to_rownames(var = "name")

##
PURB_ddw <- DESeqDataSetFromSlidingWindows(countData = PURB,
                                            colData = PURB_colData,
                                            annotObj = annotationData,
                                            tidy = FALSE,
                                            design = ~type,
                                            start0based = TRUE)

##
keep <- rowSums(counts(PURB_ddw)) >= 5
PURB_ddw <- PURB_ddw[keep,]
rm(list = "keep")

PURB_ddw <- estimateSizeFactors(PURB_ddw)
sizeFactors(PURB_ddw)

PURB_ddw_mRNAs <- PURB_ddw[ rowData(PURB_ddw)[,"gene_type"] == "protein_coding", ]
PURB_ddw_mRNAs <- estimateSizeFactors(PURB_ddw_mRNAs)
sizeFactors(PURB_ddw) <- sizeFactors(PURB_ddw_mRNAs)
sizeFactors(PURB_ddw)

PURB_ddw_tmp <- PURB_ddw
PURB_ddw_tmp <- estimateDispersions(PURB_ddw_tmp, fitType = "local", quiet = TRUE)
PURB_ddw_tmp <- nbinomWaldTest(PURB_ddw_tmp)
tmp_significant_windows <- 
  results(PURB_ddw_tmp,
          contrast = c("type", "IP", "SMI"),
          parallel = FALSE,
          BPPARAM = 4,
          tidy = TRUE,
          filterFun = ihw) %>% 
  dplyr::filter(padj < 0.05) %>% 
  .[["row"]]
rm("PURB_ddw_tmp")

PURB_ddw_mRNAs <- PURB_ddw_mRNAs[ !rownames(PURB_ddw_mRNAs) %in% tmp_significant_windows, ]
PURB_ddw_mRNAs <- estimateSizeFactors(PURB_ddw_mRNAs)
sizeFactors(PURB_ddw) <- sizeFactors(PURB_ddw_mRNAs)
sizeFactors(PURB_ddw)
rm( list = c("tmp_significant_windows", "PURB_ddw_mRNAs"))

## 
PURB_ddw <- estimateDispersions(PURB_ddw, fitType = "local", quiet = TRUE)
PURB_ddw <- nbinomWaldTest(PURB_ddw)

PURB_resultWindows <- resultsDEWSeq(PURB_ddw,
                                     contrast = c("type", "IP", "SMI"),
                                     parallel = FALSE,
                                     BPPARAM = 12,
                                     tidy = TRUE) %>% as_tibble

PURB_resultWindows[,"p_adj_IHW"] <- adj_pvalues(ihw(pSlidingWindows ~ baseMean, 
                                                     data = PURB_resultWindows,
                                                     alpha = 0.05,
                                                     nfolds = 10))

saveRDS(PURB_ddw, "./DEW-seq/Matrix_w50s20/PURB_ddw_ct5.RDS")
saveRDS(PURB_resultWindows, "./DEW-seq/Matrix_w50s20/PURB_resultWindows.RDS")

##
PURB_resultWindows %>%
  filter(p_adj_IHW < 0.05) %>%
  filter(log2FoldChange > 2) %>%
  filter(chromosome == "NC_045512.2") %>%
  arrange(desc(log2FoldChange)) %>% pull(gene_name) %>% unique() %>% head(20)

##
pdf("./Plots/DEW-seq_Dispersion_PURB_ct5.pdf", width = 7, height = 7)
plotDispEsts(PURB_ddw)
dev.off()

#extract Regions

extract_regions(resultWindows = PURB_resultWindows,
	padjThresh = 0.01,
	log2FoldChangeThresh = 2,
	trackName = "Binding Site HIV_PURB HIV 0.01",
	fileName = "./DEW-seq/Regions_w50s20/HIV_PURB_enrichedregions.log2FC_2.0_p0.01.bed") %>%
	saveRDS("./DEW-seq/Regions_w50s20/HIV_PURB_enrichedregions.log2FC_2.0_p0.01.RDS")


extract_regions(resultWindows = PURB_resultWindows,
	padjThresh = 0.05,
	log2FoldChangeThresh = 2,
	trackName = "Binding Site HIV_PURB HIV 0.05",
	fileName = "./DEW-seq/Regions_w50s20/HIV_PURB_enrichedregions.log2FC_2.0_p0.05.bed") %>%
	saveRDS("./DEW-seq/Regions_w50s20/HIV_PURB_enrichedregions.log2FC_2.0_p0.05.RDS")


# * * * * * Mock

##
Mock <- countData %>%
  dplyr::select(unique_id, contains("Mock"), -contains("IgG")) %>%
  dplyr::select(unique_id, contains("IP"), contains("SMI")) %>%
  column_to_rownames(var = "unique_id")

Mock_colData <- tibble(name = colnames(Mock)) %>%
  mutate(type = case_when(
    str_detect(name, "IP") ~ "IP",
    str_detect(name, "SMI") ~ "SMI"
  )) %>%
  mutate(type = fct_relevel(type, c("IP", "SMI"))) %>%
  column_to_rownames(var = "name")

##
Mock_ddw <- DESeqDataSetFromSlidingWindows(countData = Mock,
                                            colData = Mock_colData,
                                            annotObj = annotationData,
                                            tidy = FALSE,
                                            design = ~type,
                                            start0based = TRUE)

##
keep <- rowSums(counts(Mock_ddw)) >= 5
Mock_ddw <- Mock_ddw[keep,]
rm(list = "keep")

Mock_ddw <- estimateSizeFactors(Mock_ddw)
sizeFactors(Mock_ddw)

Mock_ddw_mRNAs <- Mock_ddw[ rowData(Mock_ddw)[,"gene_type"] == "protein_coding", ]
Mock_ddw_mRNAs <- estimateSizeFactors(Mock_ddw_mRNAs)
sizeFactors(Mock_ddw) <- sizeFactors(Mock_ddw_mRNAs)
sizeFactors(Mock_ddw)

Mock_ddw_tmp <- Mock_ddw
Mock_ddw_tmp <- estimateDispersions(Mock_ddw_tmp, fitType = "local", quiet = TRUE)
Mock_ddw_tmp <- nbinomWaldTest(Mock_ddw_tmp)
tmp_significant_windows <- 
  results(Mock_ddw_tmp,
          contrast = c("type", "IP", "SMI"),
          parallel = FALSE,
          BPPARAM = 4,
          tidy = TRUE,
          filterFun = ihw) %>% 
  dplyr::filter(padj < 0.05) %>% 
  .[["row"]]
rm("Mock_ddw_tmp")

Mock_ddw_mRNAs <- Mock_ddw_mRNAs[ !rownames(Mock_ddw_mRNAs) %in% tmp_significant_windows, ]
Mock_ddw_mRNAs <- estimateSizeFactors(Mock_ddw_mRNAs)
sizeFactors(Mock_ddw) <- sizeFactors(Mock_ddw_mRNAs)
sizeFactors(Mock_ddw)
rm( list = c("tmp_significant_windows", "Mock_ddw_mRNAs"))

## 
Mock_ddw <- estimateDispersions(Mock_ddw, fitType = "local", quiet = TRUE)
Mock_ddw <- nbinomWaldTest(Mock_ddw)

Mock_resultWindows <- resultsDEWSeq(Mock_ddw,
                                     contrast = c("type", "IP", "SMI"),
                                     parallel = FALSE,
                                     BPPARAM = 12,
                                     tidy = TRUE) %>% as_tibble

Mock_resultWindows[,"p_adj_IHW"] <- adj_pvalues(ihw(pSlidingWindows ~ baseMean, 
                                                     data = Mock_resultWindows,
                                                     alpha = 0.05,
                                                     nfolds = 10))

saveRDS(Mock_ddw, "./DEW-seq/Matrix_w50s20/Mock_ddw_ct5.RDS")
saveRDS(Mock_resultWindows, "./DEW-seq/Matrix_w50s20/Mock_resultWindows.RDS")



```



